In [1]:
import requests
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
from functools import reduce

In [7]:

def getSources():
    source_url = 'https://newsapi.org/v1/sources?language=en'
    response = requests.get(source_url).json()
    sources = []
    for source in response['sources']:
        sources.append(source['id'])
    return sources

In [8]:
def mapping():
    d = {}
    response = requests.get('https://newsapi.org/v1/sources?language=en')
    response = response.json()
    for s in response['sources']:
        d[s['id']] = s['category']
    return d



In [9]:
def category(source, m):
    try:
        return m[source]
    except:
        return 'NC'



In [10]:
def getDailyNews():
    sources = getSources()
    key = '52e22219a99b493298aa873d855fe6d9'
    url = 'https://newsapi.org/v1/articles?source={0}&sortBy={1}&apiKey={2}'
    responses = []
    for i, source in tqdm(enumerate(sources), total=len(sources)):
        
        try:
            u = url.format(source, 'top', key)
        except:
            u = url.format(source, 'latest', key)
        
        response = requests.get(u)
        r = response.json()
        try:
            for article in r['articles']:
                article['source'] = source
            responses.append(r)
        except:
            print('Rate limit exceeded ... please wait and retry in 6 hours')
            return None
                
    articles = list(map(lambda r: r['articles'], responses))
    articles = list(reduce(lambda x,y: x+y, articles))
    
    news = pd.DataFrame(articles)
    news = news.dropna()
    news = news.drop_duplicates()
    news.reset_index(inplace=True, drop=True)
    d = mapping()
    news['category'] = news['source'].map(lambda s: category(s, d))
    news['scraping_date'] = datetime.now()

    try:
        aux = pd.read_csv('C:/Users/Goldy/OneDrive/Desktop/datasets/news.csv')
        aux = aux.append(news)
        aux = aux.drop_duplicates('url')
        aux.reset_index(inplace=True, drop=True)
        aux.to_csv('C:/Users/Goldy/OneDrive/Desktop/datasets/news.csv', encoding='utf-8', index=False)
    except:
        news.to_csv('C:/Users/Goldy/OneDrive/Desktop/datasets/news.csv', index=False, encoding='utf-8')
        
    print('Done')
    


In [11]:
getDailyNews()

  0%|          | 0/51 [00:00<?, ?it/s]

Done


In [12]:
data = pd.read_csv('C:/Users/Goldy/OneDrive/Desktop/datasets/news.csv')

In [13]:
data.head()

,author,title,description,url,urlToImage,publishedAt,source,category,scraping_date
0,ABC News,Japan wins fifth straight Asia World Cup Group...,The Socceroos' traditional continental rivals ...,http://www.abc.net.au/news/2021-03-31/japan-ro...,https://live-production.wcms.abc-cdn.net.au/6e...,2021-03-30T22:00:11Z,abc-news-au,general,2021-03-31 13:53:27.159658
1,"Jesse Thompson, Kate Ashton",Push to close Kakadu's Gunlom Falls site as Pa...,A powerful land council is urged to call a spe...,http://www.abc.net.au/news/2021-03-31/push-to-...,https://live-production.wcms.abc-cdn.net.au/df...,2021-03-30T21:55:07Z,abc-news-au,general,2021-03-31 13:53:27.159658
2,"Daniel Smith, Kelsie Iorio",COVID live updates: New Queensland coronavirus...,At least a dozen new COVID-exposed locations h...,http://www.abc.net.au/news/2021-03-31/covid-li...,https://live-production.wcms.abc-cdn.net.au/57...,2021-03-30T21:44:21Z,abc-news-au,general,2021-03-31 13:53:27.159658
3,Felicity James,Santos and partners investing $4.7 billion to ...,Energy giant Santos will pour $4.7 billion int...,http://www.abc.net.au/news/2021-03-31/santos-c...,https://live-production.wcms.abc-cdn.net.au/8c...,2021-03-30T21:31:52Z,abc-news-au,general,2021-03-31 13:53:27.159658
4,ABC News,Child unknowingly tweets from US nuclear comma...,"Is ""l;;gmlxzssaw"" a US nuclear launch code? Is...",http://www.abc.net.au/news/2021-03-31/child-un...,https://live-production.wcms.abc-cdn.net.au/53...,2021-03-30T21:15:34Z,abc-news-au,general,2021-03-31 13:53:27.159658


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   author         309 non-null    object
 1   title          310 non-null    object
 2   description    293 non-null    object
 3   url            310 non-null    object
 4   urlToImage     300 non-null    object
 5   publishedAt    310 non-null    object
 6   source         310 non-null    object
 7   category       310 non-null    object
 8   scraping_date  310 non-null    object
dtypes: object(9)
memory usage: 21.9+ KB
